<a href="https://colab.research.google.com/github/dinisrferreira/Pseudodiagnosticity-in-a-continuous-learning-environment-Reis-2020-/blob/main/Reis_Pseudodiagnosticity_Model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [30]:
from google.colab import drive
drive.mount('/content/drive')

import pandas as pd
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [31]:
# Load data from Google Drive
folder_path = '/content/drive/My Drive/ProjetoGulbenkian/'
wicked_train_set = pd.read_csv(f'{folder_path}wicked_train_set.csv')
wicked_test_set = pd.read_csv(f'{folder_path}wicked_test_set.csv')
kind_train_set = pd.read_csv(f'{folder_path}kind_train_set.csv')
kind_test_set = pd.read_csv(f'{folder_path}kind_test_set.csv')
#universal_kind_train_set = pd.read_csv(f'{folder_path}universal_kind_train_set.csv')
#universal_kind_test_set = pd.read_csv(f'{folder_path}universal_kind_test_set.csv')

In [32]:
# Adjust labels to fit CrossEntropyLoss requirements
def adjust_labels(df):
    df['label'] = df['label'].map({-1: 1, 0: 0, 1: 2})
    return df

wicked_train_set = adjust_labels(wicked_train_set)
wicked_test_set = adjust_labels(wicked_test_set)
kind_train_set = adjust_labels(kind_train_set)
kind_test_set = adjust_labels(kind_test_set)
#universal_kind_train_set = adjust_labels(universal_kind_train_set)
#universal_kind_test_set = adjust_labels(universal_kind_test_set)


In [33]:
# Prepare data function
def prepare_data(df):
    X = df['input'].apply(eval).values.tolist()  # Convert string representation of list to actual list
    y = df['label'].values
    X_tensor = torch.tensor(X, dtype=torch.float32)
    y_tensor = torch.tensor(y, dtype=torch.long)
    return TensorDataset(X_tensor, y_tensor)

In [34]:
class DiagnosticNet(nn.Module):
    def __init__(self):
        super(DiagnosticNet, self).__init__()
        self.fc1 = nn.Linear(4, 10)
        self.fc2 = nn.Linear(10, 3)  # 3 classes for the 3 strategies

    def forward(self, x):
        x = torch.relu(self.fc1(x))
        x = self.fc2(x)  # Softmax will be applied in the CrossEntropyLoss
        return x

In [35]:
train_data_wicked = prepare_data(wicked_train_set)
test_data_wicked = prepare_data(wicked_test_set)
train_data_kind = prepare_data(kind_train_set)
test_data_kind = prepare_data(kind_test_set)
#train_data_universal_kind = prepare_data(universal_kind_train_set)
#test_data_universal_kind = prepare_data(universal_kind_test_set)

In [36]:
batch_size = 32
train_loader_wicked = DataLoader(train_data_wicked, batch_size=batch_size, shuffle=True)
test_loader_wicked = DataLoader(test_data_wicked, batch_size=batch_size, shuffle=False)
train_loader_kind = DataLoader(train_data_kind, batch_size=batch_size, shuffle=True)
test_loader_kind = DataLoader(test_data_kind, batch_size=batch_size, shuffle=False)
#train_loader_universal_kind = DataLoader(train_data_universal_kind, batch_size=batch_size, shuffle=True)
#test_loader_universal_kind = DataLoader(test_data_universal_kind, batch_size=batch_size, shuffle=False)

In [37]:
# Training Function
def train_model(model, train_loader, criterion, optimizer, num_epochs=50):
    for epoch in range(num_epochs):
        model.train()
        running_loss = 0.0
        for inputs, labels in train_loader:
            optimizer.zero_grad()
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()
            running_loss += loss.item()
        print(f"Epoch [{epoch+1}/{num_epochs}], Loss: {running_loss/len(train_loader):.4f}")

In [38]:
# Evaluation Function
def evaluate_model(model, test_loader):
    model.eval()
    correct = 0
    total = 0
    with torch.no_grad():
        for inputs, labels in test_loader:
            outputs = model(inputs)
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
    accuracy = 100 * correct / total
    print(f"Accuracy: {accuracy:.2f}%")

In [39]:
# Initialize the model, criterion, and optimizer
model_wicked = DiagnosticNet()
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model_wicked.parameters(), lr=0.001)

In [40]:
# Train the model
train_model(model_wicked, train_loader_wicked, criterion, optimizer, num_epochs=50)

Epoch [1/50], Loss: 1.0855
Epoch [2/50], Loss: 1.0508
Epoch [3/50], Loss: 1.0161
Epoch [4/50], Loss: 0.9826
Epoch [5/50], Loss: 0.9499
Epoch [6/50], Loss: 0.9184
Epoch [7/50], Loss: 0.8889
Epoch [8/50], Loss: 0.8612
Epoch [9/50], Loss: 0.8363
Epoch [10/50], Loss: 0.8140
Epoch [11/50], Loss: 0.7957
Epoch [12/50], Loss: 0.7799
Epoch [13/50], Loss: 0.7664
Epoch [14/50], Loss: 0.7546
Epoch [15/50], Loss: 0.7445
Epoch [16/50], Loss: 0.7355
Epoch [17/50], Loss: 0.7273
Epoch [18/50], Loss: 0.7200
Epoch [19/50], Loss: 0.7132
Epoch [20/50], Loss: 0.7076
Epoch [21/50], Loss: 0.7017
Epoch [22/50], Loss: 0.6968
Epoch [23/50], Loss: 0.6928
Epoch [24/50], Loss: 0.6880
Epoch [25/50], Loss: 0.6839
Epoch [26/50], Loss: 0.6803
Epoch [27/50], Loss: 0.6768
Epoch [28/50], Loss: 0.6736
Epoch [29/50], Loss: 0.6704
Epoch [30/50], Loss: 0.6678
Epoch [31/50], Loss: 0.6651
Epoch [32/50], Loss: 0.6626
Epoch [33/50], Loss: 0.6604
Epoch [34/50], Loss: 0.6582
Epoch [35/50], Loss: 0.6561
Epoch [36/50], Loss: 0.6543
E

In [43]:
# Initialize the model, criterion, and optimizer
model_wicked = DiagnosticNet()
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model_wicked.parameters(), lr=0.001)

# Train the model
train_model(model_wicked, train_loader_wicked, criterion, optimizer, num_epochs=150)

# Evaluate the model
evaluate_model(model_wicked, test_loader_wicked)

Epoch [1/150], Loss: 1.0911
Epoch [2/150], Loss: 1.0506
Epoch [3/150], Loss: 1.0105
Epoch [4/150], Loss: 0.9705
Epoch [5/150], Loss: 0.9310
Epoch [6/150], Loss: 0.8935
Epoch [7/150], Loss: 0.8578
Epoch [8/150], Loss: 0.8263
Epoch [9/150], Loss: 0.7990
Epoch [10/150], Loss: 0.7760
Epoch [11/150], Loss: 0.7555
Epoch [12/150], Loss: 0.7387
Epoch [13/150], Loss: 0.7232
Epoch [14/150], Loss: 0.7111
Epoch [15/150], Loss: 0.7006
Epoch [16/150], Loss: 0.6926
Epoch [17/150], Loss: 0.6846
Epoch [18/150], Loss: 0.6779
Epoch [19/150], Loss: 0.6725
Epoch [20/150], Loss: 0.6670
Epoch [21/150], Loss: 0.6625
Epoch [22/150], Loss: 0.6585
Epoch [23/150], Loss: 0.6551
Epoch [24/150], Loss: 0.6515
Epoch [25/150], Loss: 0.6489
Epoch [26/150], Loss: 0.6459
Epoch [27/150], Loss: 0.6437
Epoch [28/150], Loss: 0.6418
Epoch [29/150], Loss: 0.6391
Epoch [30/150], Loss: 0.6367
Epoch [31/150], Loss: 0.6352
Epoch [32/150], Loss: 0.6327
Epoch [33/150], Loss: 0.6308
Epoch [34/150], Loss: 0.6290
Epoch [35/150], Loss: 0